In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# plt.plot(solar_output_db.loc[0,"solar_output_list"][:87600])

In [ ]:
# Reading solar output dataset from json file
solar_output_db = pd.read_json(r'C:\Users\codingPurposes\Desktop\rawData\solar_output\merged_lean_solar_data.json',orient='split')

# Dropping unnecessary columns and renaming the remaining one
solar_output = solar_output_db.drop(['lat', 'lon', "gemeentenaam","population", "solar_data_list", "temperature_data_list"], axis=1)
solar_output = solar_output.rename(index=str, columns={"solar_output_list": "data"})

# Restructuring the dataframe in such a way that rows become columns
solar_output_muniInColumns = solar_output[['data']].unstack().apply(pd.Series).T
solar_output_muniInColumns = solar_output_muniInColumns.xs('data', axis=1, drop_level=True)

avg_solar_output = pd.DataFrame(columns=["AvgValue"])
avg_solar_output["AvgValue"] = solar_output_muniInColumns.loc[:,:].mean(axis=1)

avg_solar_output.to_json(r'C:\Users\codingPurposes\Desktop\rawData\solar_output\mean_solar_output_hourly_10years.json', orient='split')

In [ ]:
ax = avg_solar_output.loc[:8760, :].plot(figsize=[16,6])
ax.set_xlabel("Hours", fontsize= 14)
ax.set_ylabel("Solar Output (KWh)", fontsize= 14)
plt.title("Average solar output of the Netherlands in 2020", {'fontsize': 18})
fig = ax.get_figure()
fig.savefig(r'C:\Users\codingPurposes\Desktop\for_report\solaroutput2020.png')

In [ ]:
ax = avg_solar_output.loc[0:24, :].plot(figsize=[16,6])
ax.set_xlabel("Hours", fontsize= 14)
ax.set_ylabel("Solar Output (KWh)", fontsize= 14)
plt.title("Daily profile of average solar output", {'fontsize': 18})
fig = ax.get_figure()
fig.savefig(r'C:\Users\codingPurposes\Desktop\for_report\solaroutputDaily.png')

In [ ]:
plt.plot(solar_output_db.loc[0, "solar_data_list"][48:72])

In [ ]:
solar_output_db.loc[0, "solar_output_list"]

### Checkpoint - 1

In [5]:
avg_solar_output = pd.read_json(r'C:\Users\codingPurposes\Desktop\rawData\solar_output\mean_solar_output_hourly_10years.json', orient='split')

In [6]:
# Extending data set to 30 years:
avg_solar_output = pd.concat([avg_solar_output, avg_solar_output, avg_solar_output], ignore_index=True)

In [7]:
for d in tqdm(range(3650*3)):
    for h in range(24):
        avg_solar_output.loc[(d*24)+h, "dayIndex"] = d

representativeDays = [149, 336, 1255, 1381, 1414, 1486, 2239, 3284, 3462]
listHours = []

for y in range(3):
    for d in representativeDays:
        for h in range(24):
            listHours.append(24*(d+y*3650) + h)

avg_solar_output = avg_solar_output[avg_solar_output.index.isin(listHours)]
avg_solar_output = avg_solar_output.reset_index(drop=True)

100%|██████████| 10950/10950 [00:46<00:00, 236.67it/s]


In [8]:
three_hourly_solar_output = pd.DataFrame(columns=["Value"])

emptyListValue = []
emptyListDay = []

for i in tqdm(range(int(len(avg_solar_output)/3))):
    day = avg_solar_output.loc[1+(3*i),"dayIndex"]
    value = avg_solar_output.loc[1+(3*i),"AvgValue"] * 3
    emptyListValue.append(value)
    emptyListDay.append(day)
    
three_hourly_solar_output["Value"] = emptyListValue
three_hourly_solar_output["dayIndex"] = emptyListDay


100%|██████████| 216/216 [00:00<00:00, 53968.53it/s]


In [9]:
daily_solar_output = pd.DataFrame(columns=["Value", "dayIndex"])
for i in tqdm(range(int(len(three_hourly_solar_output)/8))):
    daily_solar_output.loc[i, "dayIndex"] = three_hourly_solar_output.loc[(0 + 8*i), "dayIndex"]
    daily_solar_output.loc[i, "Value"] = list(three_hourly_solar_output.loc[(0 + 8*i):(7 + 8*i), "Value"])

100%|██████████| 27/27 [00:00<00:00, 793.30it/s]


In [ ]:
daily_solar_output.to_json(r"C:\Users\codingPurposes\Desktop\processedData\processed_solar_output\daily_3hourly_SolarOutput.json", orient='split')

In [ ]:
# three_hourly_solar_output = pd.DataFrame(columns=["Value"])
# emptyList = []

# for i in tqdm(range(int(len(avg_solar_output)/3))):
    
#     value = avg_solar_output.loc[1+(3*i),"AvgValue"] * 3
#     emptyList.append(value)

# three_hourly_solar_output["Value"] = emptyList

In [ ]:
# three_hourly_solar_output_2years = pd.DataFrame(columns=["Value"])
# emptyList = []

# for i in tqdm(range(15)):
#     for t in range(2920):
#         value = three_hourly_solar_output.loc[(i*2*2920)+t, "Value"]
#         emptyList.append(value)
        
# three_hourly_solar_output_2years["Value"] = emptyList

### Creating daily data set for 3 hourly solar output values:

In [ ]:
# daily_solar_output = pd.DataFrame(columns=["Value"])
# emptyList = []

# for i in tqdm(range(int(len(three_hourly_solar_output_2years)/8))):
#     value = list(three_hourly_solar_output_2years.loc[(0 + 8*i):(7 + 8*i), "Value"])
#     emptyList.append(value)
    
# daily_solar_output["Value"] = emptyList

# three_hourly_solar_output_2years.to_json(r"C:\Users\codingPurposes\Desktop\processedData\processed_solar_output\three_hourly_solar_output_2years.json", orient='split')
# daily_solar_output.to_json(r"C:\Users\codingPurposes\Desktop\processedData\processed_solar_output\daily_3hourly_SolarOutput_2years_identical.json", orient='split')